# Task 1
## 2

Next, take the two well-known datasets: Fashion MNIST (introduced in Ch 10, p.  298) and CIFAR-10.The first dataset contains 2D (grayscale) images of size 28x28, split into 10 categories; 60,000 images for training  and  10,000  for  testing,  while  the  latter  contains  32x32x3  RGB  images  (50,000/10,000train/test). Apply two reference networks on the fashion MNIST dataset

### b
A convolutional neural network

In [4]:
# Importing libraries
import tensorflow as tf
from tensorflow import keras
import sklearn
from sklearn.datasets import load_sample_image
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [5]:
# Loading the data
fashion_mnist = keras.datasets.fashion_mnist

In [21]:
# Creating the train and test sets
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

# Check the dimensions 
batch_size, height, width = X_train_full.shape
print(batch_size, height, width)
input_shape = (height, width, 1) #because 1 channel

# As it only has 1 channel we want to reshape it for symmetry
X_train_full = X_train_full.reshape(batch_size, height, width, 1)
X_test = X_test.reshape(X_test.shape[0], height, width, 1)

# Normalising the pixel values to the range [0, 1]
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0

# Creating a validation set of 10%
validation_size = int(0.1 * len(X_train_full))

X_valid, X_train = X_train_full[:validation_size], X_train_full[validation_size:]
y_valid, y_train = y_train_full[:validation_size], y_train_full[validation_size:]

60000 28 28


In [23]:
# For more efficient code we"ll change the data types to float32
X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')
X_test = X_test.astype('float32')

In [26]:
# Lets first try the model from the book
model = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation="relu", padding="same",
                        input_shape = input_shape), # No stride because images are not very large
    keras.layers.MaxPooling2D(2), # divides each spatial dimension by 2
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"), # double filters
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2), # divide dimensions by 2
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"), # again double filters
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2), 
    # fully connected network:
    keras.layers.Flatten(), # needs to be 1D for dense network
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5), # droupout of 50% to prevent overfitting
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax") # output layer
])

C:\Users\hsuik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [28]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [30]:
model.fit(X_train, y_train, batch_size=64, epochs=12, validation_data = (X_valid, y_valid))

Epoch 1/12
844/844 ━━━━━━━━━━━━━━━━━━━━ 153s 181ms/step - accuracy: 0.6105 - loss: 1.0420 - val_accuracy: 0.7513 - val_loss: 0.6913
Epoch 2/12
844/844 ━━━━━━━━━━━━━━━━━━━━ 164s 195ms/step - accuracy: 0.6803 - loss: 0.8617 - val_accuracy: 0.7772 - val_loss: 0.5991
Epoch 3/12
844/844 ━━━━━━━━━━━━━━━━━━━━ 205s 243ms/step - accuracy: 0.7248 - loss: 0.7464 - val_accuracy: 0.8045 - val_loss: 0.5210
Epoch 4/12
844/844 ━━━━━━━━━━━━━━━━━━━━ 205s 243ms/step - accuracy: 0.7537 - loss: 0.6771 - val_accuracy: 0.8205 - val_loss: 0.4922
Epoch 5/12
844/844 ━━━━━━━━━━━━━━━━━━━━ 225s 267ms/step - accuracy: 0.7758 - loss: 0.6168 - val_accuracy: 0.8308 - val_loss: 0.4667
Epoch 6/12
844/844 ━━━━━━━━━━━━━━━━━━━━ 256s 304ms/step - accuracy: 0.7914 - loss: 0.5843 - val_accuracy: 0.8427 - val_loss: 0.4333
Epoch 7/12
844/844 ━━━━━━━━━━━━━━━━━━━━ 152s 181ms/step - accuracy: 0.8012 - loss: 0.5542 - val_accuracy: 0.8408 - val_loss: 0.4258
Epoch 8/12
844/844 ━━━━━━━━━━━━━━━━━━━━ 176s 208ms/step - accuracy: 0.8134 -

In [31]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.39081478118896484
Test accuracy: 0.857699990272522
